# Land-use forcing

Smith et al. 2020:

- 2014 - 1850 land use forcing of -0.18 W m-2

The time series that scales with AFOLU is our best bet.

In [1]:
import os

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from scipy.optimize import root

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

/Users/c.l.b.alexander/Documents/FAIR/fair_virtual_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [3]:
target_forcing = -0.18

scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
configs = ['UKESM1-0-LL']
species = ['CO2 AFOLU', 'Land use']

In [4]:
def erf_rootfinder(x):
    f = FAIR()
    f.define_time(1850, 2101, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(*read_properties(species=species, filename='../data/species_configs_properties_vanilla.csv'))
    f.allocate()
    f.fill_from_rcmip()
#    f.fill_species_configs()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    fill(f.species_configs['land_use_cumulative_emissions_to_forcing'], x, specie='CO2 AFOLU')
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)  # necessary
    f.run(progress=False)
    return f.forcing[165, 0, 0, 1] - target_forcing

In [5]:
sol = root(erf_rootfinder, -0.006)

In [6]:
sol.x[0]

np.float64(-0.0003024895261521421)

In [7]:
f = FAIR()
f.define_time(1850, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(*read_properties(species=species, filename='../data/species_configs_properties_vanilla.csv'))
f.allocate()
f.fill_from_rcmip()
#    f.fill_species_configs()
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
fill(f.species_configs['land_use_cumulative_emissions_to_forcing'], sol.x[0], specie='CO2 AFOLU')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)  # necessary
f.run()

Running 4 projections in parallel: 100%|██████████| 251/251 [00:00<00:00, 42001.61timesteps/s]


In [8]:
f.forcing[:,:,0,-1]

<xarray.DataArray (timebounds: 252, scenario: 4)> Size: 8kB
array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [-0.00055701, -0.00055701, -0.00055701, -0.00055701],
       [-0.00110531, -0.00110531, -0.00110531, -0.00110531],
       ...,
       [-0.15206705, -0.16758574, -0.25883627, -0.21863813],
       [-0.15121476, -0.16614838, -0.25963696, -0.21820653],
       [-0.15033783, -0.1646964 , -0.26044041, -0.21774362]],
      shape=(252, 4))
Coordinates:
  * timebounds  (timebounds) float64 2kB 1.85e+03 1.851e+03 ... 2.101e+03
  * scenario    (scenario) <U6 96B 'ssp126' 'ssp245' 'ssp370' 'ssp585'
    config      <U11 44B 'UKESM1-0-LL'
    specie      <U9 36B 'Land use'

In [9]:
os.makedirs('../data/calibration/', exist_ok=True)

In [10]:
landuse_erf = f.forcing[:,:,0,-1].to_pandas()
# landuse_erf.name = 'landuse_erf'
landuse_erf.to_csv('../data/calibration/landuse_forcing.csv')

In [11]:
# landuse_erf